## 2章　よく利用される確率分布

<a href="https://colab.research.google.com/github/makaishi2/python_bayes_intro/blob/main/notebooks/2%E7%AB%A0_%E3%82%88%E3%81%8F%E5%88%A9%E7%94%A8%E3%81%95%E3%82%8C%E3%82%8B%E7%A2%BA%E7%8E%87%E5%88%86%E5%B8%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### 共通処理

In [ ]:
%matplotlib inline
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [ ]:
# ライブラリのimport

# NumPy用ライブラリ
import numpy as np
# Matplotlib中のpyplotライブラリのインポート
import matplotlib.pyplot as plt
# matplotlib日本語化対応ライブラリのインポート
import japanize_matplotlib
# pandas用ライブラリ
import pandas as pd
# データフレーム表示用関数
from IPython.display import display
# seaborn
import seaborn as sns
# 表示オプション調整
# NumPy表示形式の設定
np.set_printoptions(precision=3, floatmode='fixed')
# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14
# サイズ設定
plt.rcParams['figure.figsize'] = (6, 6)
# 方眼表示ON
plt.rcParams['axes.grid'] = True
# データフレームでの表示精度
pd.options.display.float_format = '{:.3f}'.format
# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

In [ ]:
import pymc as pm
import arviz as az

print(f"Running on PyMC v{pm.__version__}")
print(f"Running on ArViz v{az.__version__}")

### 2.1 ベルヌーイ分布

#### 確率モデル定義

In [ ]:
# パラメータ設定
p = 0.5

model1 = pm.Model()
with model1:
    # pm.Bernoulli: ベルヌーイ分布
    # p: くじに当たる確率
    x = pm.Bernoulli('x', p=p)

#### サンプリング

In [ ]:
with model1:
    prior_samples1 = pm.sample_prior_predictive(random_seed=42)

#### 結果分析

##### NumPy形式のサンプル値抽出

In [ ]:
x_samples1 = prior_samples1['prior']['x'].values
print(x_samples1)

##### サンプリング結果の統計分析

In [ ]:
summary1 = az.summary(prior_samples1, kind='stats')
display(summary1)

  ##### サンプリング結果の可視化

In [ ]:
ax = az.plot_dist(x_samples1)
ax.set_title(f'ベルヌーイ分布　p={p}');

### 2.2 二項分布

#### 確率モデル定義(p=0.5, n=5)

In [ ]:
# パラメータ設定
p = 0.5
n = 5

model2 = pm.Model()
with model2:
    # pm.Binomial: 二項分布
    # p: くじに当たる確率
    # n: 試行回数
    x = pm.Binomial('x', p=p, n=n)

#### サンプルリングと結果分析

In [ ]:
with model2:
    # サンプリング
    prior_samples2 = pm.sample_prior_predictive(random_seed=42)

# サンプル値抽出
x_samples2 = prior_samples2['prior']['x'].values
print(x_samples2)

# サンプリング結果の統計分析
summary2 = az.summary(prior_samples2, kind='stats')
display(summary2)

# サンプリング結果の可視化
ax = az.plot_dist(x_samples2)
ax.set_title(f'二項分布　p={p} n={n}');

#### 確率モデル定義(p=0.5, n=50)、サンプリング

In [ ]:
# パラメータ設定
p = 0.5
n = 50

model3 = pm.Model()
with model3:
    # pm.Binomial: 二項分布
    # p: くじに当たる確率
    # n: 試行回数
    x = pm.Binomial('x', p=p, n=n)

    # サンプリング
    prior_samples3 = pm.sample_prior_predictive(random_seed=42)

# サンプル値の抽出
x_samples3 = prior_samples3['prior']['x'].values
print(x_samples3)

# サンプリング結果の統計分析
summary3 = az.summary(prior_samples3, kind='stats')
display(summary3)

# サンプリング結果の可視化
ax = az.plot_dist(x_samples3)
ax.set_title(f'二項分布　p={p} n={n}');

### 2.3 正規分布

#### アイリスデータセットからsetosaのがく片長(sepal_length)の分布を調べる

In [ ]:
# アイリスデータセットの読み込み
df = sns.load_dataset('iris')

# setosaの行のみ抽出
df1 = df.query('species == "setosa"')

bins = np.arange(4.0, 6.2, 0.2)
# 分布の確認
sns.histplot(df1, x='sepal_length', bins=bins, kde=True)
plt.xticks(bins);

#### 正規分布関数のグラフ

In [ ]:
# 正規分布関数の定義
# 式(2.3)をNumPyで実装
def norm(x, mu, sigma):
    return np.exp(-((x - mu)/sigma)**2/2) / (np.sqrt(2 * np.pi) * sigma)

# パラメータ定義
mu1, sigma1 = 3.0, 2.0
mu2, sigma2 = 1.0, 3.0

# グラフ描画用x座標の定義
# 2つの正規分布関数で±3sigmaまで入るように計算
x = np.arange(-8.0, 10.0, 0.01)

# x軸目盛の設定
xticks = np.arange(-8.0, 11.0, 1.0)

# グラフ描画
plt.plot(x, norm(x, mu1, sigma1), label=f'mu={mu1} sigma={sigma1}')
plt.plot(x, norm(x, mu2, sigma2), label=f'mu={mu2} sigma={sigma2}')
plt.xticks(xticks, fontsize=12)
plt.legend(fontsize=12)
plt.title(f'正規分布関数のグラフ');

#### 確率モデル定義(mu=0.0 sigma=1.0)

In [ ]:
# パラメータ設定
mu = 0.0
sigma = 1.0

model4 = pm.Model()
with model4:
    # pm.Normal: 正規分布
    # mu:平均
    # sigma:標準偏差
    x = pm.Normal('x', mu=mu, sigma=sigma)

#### サンプリング

In [ ]:
with model4:
    # サンプリング
    prior_samples4 = pm.sample_prior_predictive(random_seed=42)

# サンプル値抽出
x_samples4 = prior_samples4['prior']['x'].values
# 桁数が多いので先頭100個だけに限定
print(x_samples4[:,:100])

# サンプリング結果の可視化
ax = az.plot_dist(x_samples4)
ax.set_title(f'正規分布　mu={mu} sigma={sigma}');

#### ヒストグラムによる表示

In [ ]:
bins = np.arange(-3,3.5,0.5)
ax = az.plot_dist(x_samples4, kind='hist',
    hist_kwargs={'bins':bins})
plt.xticks(np.arange(-3,4,1))
ax.set_title(f'正規分布　mu={mu} sigma={sigma}');

#### サンプリング結果の統計分析

In [ ]:
summary4 = az.summary(prior_samples4, kind='stats')
display(summary4)


#### 確率モデル定義(mu=3.0 sigma=2,0)とサンプリング結果分析


In [ ]:
# パラメータ設定
mu = 3.0
sigma = 2.0

model5 = pm.Model()
with model5:
    # pm.Normal: 正規分布
    # mu:平均
    # sigma:標準偏差
    x = pm.Normal('x', mu=mu, sigma=sigma)

    # サンプリング
    prior_samples5 = pm.sample_prior_predictive(random_seed=42)

# サンプル値抽出
x_samples5 = prior_samples5['prior']['x'].values
# 桁数が多いので先頭100個だけに限定
print(x_samples5[:,:100])

# サンプリング結果の可視化
ax = az.plot_dist(x_samples5)
ax.set_title(f'正規分布　mu={mu} sigma={sigma}');

#### ヒストグラムによる表示

In [ ]:
bins = np.arange(-4, 10, 1.0)
az.plot_dist(x_samples5, kind='hist',
    hist_kwargs={'bins':bins})
plt.title(f'正規分布　mu={mu} sigma={sigma}');

#### サンプリング結果の統計分析

In [ ]:
summary5 = az.summary(prior_samples5, kind='stats')
display(summary5)

### 2.4 一様分布

#### 確率モデル生成(lower=0.0, upper=1.0)

In [ ]:
# パラメータ設定
lower = 0.0
upper = 1.0

model6 = pm.Model()
with model6:
    # pm.Uniform: 一様分布
    # lower: 下限値
    # upper: 上限値
    x = pm.Uniform('x', lower=lower, upper=upper)

#### サンプリングと結果分析

In [ ]:
with model6:
    # サンプリング
    prior_samples6 = pm.sample_prior_predictive(random_seed=42)

# サンプル値抽出
x_samples6 = prior_samples6['prior']['x'].values
print(x_samples6[:,:100])

# サンプリング結果の可視化
ax = az.plot_dist(x_samples6)
ax.set_title(f'一様分布 lower={lower} upper={upper}');

#### サンプリング結果のヒストグラム表示

In [ ]:
bins = np.arange(0.0,1.1,0.1)
ax = az.plot_dist(x_samples6, kind='hist',
    hist_kwargs={'bins':bins})
ax.set_title(f'一様分布 lower={lower} upper={upper}');

#### サンプリング結果の統計分析

In [ ]:
summary6 = az.summary(prior_samples6, kind='stats')
display(summary6)

#### 確率モデル定義(下限値0.1、上限値0.9)とサンプリング結果分析

In [ ]:
# パラメータ設定
lower = 0.1
upper = 0.9

model7 = pm.Model()
with model7:
    # pm.Uniform: 一様分布
    # lower: 下限値
    # upper: 上限値
    x = pm.Uniform('x', lower=lower, upper=upper)

    # サンプリング
    prior_samples7 = pm.sample_prior_predictive(random_seed=42)

# サンプル値抽出
x_samples7 = prior_samples7['prior']['x'].values
print(x_samples7[:,:100])

# サンプリング結果の可視化
ax = az.plot_dist(x_samples7)
ax.set_title(f'一様分布 lower={lower} upper={upper}');

#### サンプリング結果のヒストグラム表示

In [ ]:
bins = np.arange(0.0,1.1,0.1)
ax = az.plot_dist(x_samples7, kind='hist',
    hist_kwargs={'bins':bins})
ax.set_title(f'一様分布 lower={lower} upper={upper}');

### 2.5 ベータ分布

#### ベータ関数のグラフ (alpha=3, beta=4)

In [ ]:
from math import gamma

# パラメータ定義
alpha = 2.0 + 1.0
beta = 3.0 + 1.0

# ベータ関数の定義
def Beta(p, alpha, beta):
    C = gamma(alpha+beta-1)/(gamma(alpha-1)*gamma(beta-1))
    return C * p ** (alpha-1) * (1-p) ** (beta-1)

# グラフ描画用x座標の定義
# 0.0 < p < 1.0
p = np.arange(0.0, 1.0, 0.01)

# グラフ描画
plt.plot(p, Beta(p, alpha, beta))
plt.title(f'ベータ関数 alpha={alpha} beta={beta}')
plt.xlabel(r'$p$')
plt.ylabel('尤度');

#### 確率モデル定義(alpha=3, beta=4)

In [ ]:
# パラメータ設定
alpha = 2.0 + 1.0
beta = 3.0 + 1.0

model8 = pm.Model()
with model8:
    # pm.Beta: ベータ分布
    # alpha:くじに当たった回数+1
    # beta:くじにはずれた回数+1
    p = pm.Beta('p', alpha=alpha, beta=beta)

#### サンプリングと結果分析

In [ ]:
with model8:
    # サンプリング
    prior_samples8 = pm.sample_prior_predictive(random_seed=42)

# サンプル値抽出
p_samples8 = prior_samples8['prior']['p'].values
# 桁数が多いので先頭100個だけに限定
print(p_samples8[:,:100])

# サンプリング結果の可視化
ax = az.plot_dist(p_samples8)
ax.set_title(f'ベータ分布　alpha={alpha} beta={beta}');

#### サンプル結果のヒストグラム表示

In [ ]:
bins = np.arange(0, 1.0, 0.1)
ax = az.plot_dist(p_samples8, kind='hist',
    hist_kwargs={'bins':bins})
ax.set_title(f'ベータ分布　alpha={alpha} beta={beta}');

#### サンプリング結果の統計分析

In [ ]:
summary8 = az.summary(prior_samples8, kind='stats')
display(summary8)

#### 確率モデル定義(alpha=21, beta=31)とサンプリング結果分析

In [ ]:
# パラメータ設定
alpha = 20.0 + 1.0
beta = 30.0 + 1.0

model9 = pm.Model()
with model9:
    # pm.Beta: ベータ分布
    # alpha: 注目している試行の成功数+1
    # beta: 注目している試行の失敗数+1
    p = pm.Beta('p', alpha=alpha, beta=beta)

    # サンプリング
    prior_samples9 = pm.sample_prior_predictive(random_seed=42)

# サンプル値抽出
p_samples9 = prior_samples9['prior']['p'].values
# 桁数が多いので先頭100個だけに限定
print(p_samples9[:,:100])

# サンプリング結果の可視化
ax = az.plot_dist(p_samples9)
ax.set_title(f'ベータ分布　alpha={alpha} beta={beta}');

####  サンプリング結果のヒストグラム表示

In [ ]:
bins = np.arange(0.2, 0.6, 0.05)
ax = az.plot_dist(p_samples9, kind='hist',
    hist_kwargs={'bins':bins})
ax.set_title(f'ベータ分布　alpha={alpha} beta={beta}');

#### サンプリング結果の統計分析

In [ ]:
summary9 = az.summary(prior_samples9, kind='stats')
display(summary9)

### 2.6 半正規分布


#### 確率モデル定義(sigma=1.0)

In [ ]:
# パラメータ設定
sigma = 1.0

model10 = pm.Model()
with model10:
    # pm.HalfNormal: 半正規分布
    # sigma: 標準偏差
    x = pm.HalfNormal('x', sigma=sigma)

#### サンプリングと結果分析

In [ ]:
with model10:
    # サンプリング
    prior_samples10 = pm.sample_prior_predictive(random_seed=42)

# サンプル値抽出
x_samples10 = prior_samples10['prior']['x'].values
# 桁数が多いので先頭100個だけに限定
print(x_samples10[:,:100])

# サンプリング結果の可視化
ax = az.plot_dist(x_samples10)
ax.set_title(f'半正規分布　sigma={sigma}');

### コラム　HDIとCIの違い

 #### CIグラフ描画準備

In [ ]:
# カイ2乗分布(nu=3)の確率密度関数
def qui_square(x):
    w = x/(2*np.pi)
    return np.sqrt(w) * np.exp(-x/2)

# x1, x2はa1, a2の値が0.1になるようにヒューリスティックに求めた
x1 = 0.585
x2 = 6.25

# 数値積分でグラフの面積を計算
from scipy import integrate
a1, _err1 = integrate.quad(qui_square, 0, x1)
a2, _err2 = integrate.quad(qui_square, x2, 1000)
a = np.array([a1, a2])
print(a)

#### CIグラフ描画

In [ ]:
x = np.arange(0.0, 15, 0.01)
x_between = (x1 < x) & (x  < x2)
y = qui_square(x)
plt.plot(x, y)
plt.fill_between(x, 0, y, where=x_between, color='b')
plt.plot((x1, x1), (0, 0.25), c='k')
plt.plot((x2, x2), (0, 0.25), c='k')
plt.title('CI (confidence interval)の考え方');

#### HDIグラフ描画準備

In [ ]:
from scipy import optimize

def h(x):
    return qui_square(x) - 0.05

print(optimize.fsolve(h, 0.0))
print(optimize.fsolve(h, 5.0))

#### HDIグラフ描画

In [ ]:
x = np.arange(0.0, 15, 0.01)
x1 = 0.02
x2 = 5.93
y = qui_square(x)
plt.plot(x, y)
plt.fill_between(x, 0.0, y, where= np.logical_and(x > x1, x < x2), color='b')
plt.plot((0, 15.0), (0.05, 0.05), c='k')
plt.plot((x1, x1), (0, 0.25), c='k')
plt.plot((x2, x2), (0, 0.25), c='k')
plt.title('HDI (highest density interval)の考え方');

### バージョンの確認

In [ ]:
!pip install watermark | tail -n 1
%load_ext watermark
%watermark --iversions